In [1]:
print('Imports')
!pip3 install tensorflow==1.13.1

import os
import io, sys, random
from tensorflow import keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding


Imports
     |████████████████████████████████| 92.5MB 34kB/s 
     |████████████████████████████████| 368kB 47.4MB/s 
     |████████████████████████████████| 3.2MB 44.0MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print('test')
!curl https://raw.githubusercontent.com/dexterfichuk/GoogleDriveCheckpoint/master/google_drive_checkpoint.py -O
print('test2')
from google_drive_checkpoint import GoogleDriveCheckpoint



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

test
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4522  100  4522    0     0  33007      0 --:--:-- --:--:-- --:--:-- 33007
test2


Using TensorFlow backend.


In [0]:
MIN_WORD_FREQUENCY = os.environ.get('MIN_WORD_FREQUENCY', 10)
SEQUENCE_LEN = os.environ.get('SEQUENCE_LEN', 30)
USE_DROPOUT = os.environ.get('METLRNN_USE_DROPOUT', True)
DROPOUT = os.environ.get('METLRNN_DROPOUT', 0.2)
BATCH_SIZE = os.environ.get('METLRNN_BATCH_SIZE', 4098*2)

In [0]:
def corpus_to_dictionary(path: str):
    print('corupus to dict')
    with io.open(path) as f:
        # Get words from corpus file
        text = f.read().lower().replace('\n', ' \n ').replace('\\', ' \n ')
        text_in_words = keras.preprocessing.text.text_to_word_sequence(text, filters='"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t',
                                                                       lower=True,
                                                                       split=' ')
        print('Corpus length in words:', len(text_in_words))
        if ('\n' in text_in_words):
            index = text_in_words.index('\n')
            test = text_in_words[index]

        # Count how many times word appears in text_in_words
        word_freq = {}
        for word in text_in_words:
            word_freq[word] = word_freq.get(word, 0) + 1

        # Get ignored words and add them to ignored_words_set
        ignored = set()
        for k, v in word_freq.items():
            if word_freq[k] < MIN_WORD_FREQUENCY:
                ignored.add(k)

        words = set(text_in_words)
        print('Unique words:', len(words))

        # Remove ignored words from set
        words = sorted(set(words) - ignored)
        print('Unique words after removing ignored words:', len(words))

        # Create two dictionaries. One with word as a key and index as value. One with index as key and word as a value
        word_indices = dict((c, i) for i, c in enumerate(words))
        indices_word = dict((i, c) for i, c in enumerate(words))

        print('EOF: corpus_to_dictionary()')
        return text_in_words, ignored, word_indices, indices_word


def create_and_filter_sequences(text_in_words, ignored_words):
    print('start: create_and_filter_sequences')
    STEP = 1
    sentences = []
    next_words = []
    ignored = 0

    # Loop original corpus. Add SEQUENCES_LEN long sentences to sentences and SEQUENCES_LEN next words to next_words
    # Only add sentences that don't contain ignored words
    for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
        # Only add sequences where no word is in ignored_words
        if len(set(text_in_words[i: i + SEQUENCE_LEN + 1]).intersection(ignored_words)) == 0:
            sentences.append(text_in_words[i: i + SEQUENCE_LEN])
            next_words.append(text_in_words[i + SEQUENCE_LEN])
        else:
            ignored = ignored + 1
    print('Ignored sequences:', ignored)
    print('Remaining sequences:', len(sentences))

    return sentences, next_words


def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []

    for i in np.random.RandomState(seed=42).permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1. - (percentage_test / 100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))

    print('end: shuffle_and_split_training_set')

    return (x_train, y_train), (x_test, y_test)

Data tools

In [0]:
def corpus_to_dictionary(path: str):
    print('corupus to dict')
    with io.open(path) as f:
        # Get words from corpus file
        text = f.read().lower().replace('\n', ' \n ').replace('\\', ' \n ')
        text_in_words = keras.preprocessing.text.text_to_word_sequence(text, filters='"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t',
                                                                       lower=True,
                                                                       split=' ')
        print('Corpus length in words:', len(text_in_words))
        if ('\n' in text_in_words):
            index = text_in_words.index('\n')
            test = text_in_words[index]

        # Count how many times word appears in text_in_words
        word_freq = {}
        for word in text_in_words:
            word_freq[word] = word_freq.get(word, 0) + 1

        # Get ignored words and add them to ignored_words_set
        ignored = set()
        for k, v in word_freq.items():
            if word_freq[k] < MIN_WORD_FREQUENCY:
                ignored.add(k)

        words = set(text_in_words)
        print('Unique words:', len(words))

        # Remove ignored words from set
        words = sorted(set(words) - ignored)
        print('Unique words after removing ignored words:', len(words))

        # Create two dictionaries. One with word as a key and index as value. One with index as key and word as a value
        word_indices = dict((c, i) for i, c in enumerate(words))
        indices_word = dict((i, c) for i, c in enumerate(words))

        print('EOF: corpus_to_dictionary()')
        return text_in_words, ignored, word_indices, indices_word, words


def create_and_filter_sequences(text_in_words, ignored_words):
    print('start: create_and_filter_sequences')
    STEP = 1
    sentences = []
    next_words = []
    ignored = 0

    # Loop original corpus. Add SEQUENCES_LEN long sentences to sentences and SEQUENCES_LEN next words to next_words
    # Only add sentences that don't contain ignored words
    for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
        # Only add sequences where no word is in ignored_words
        if len(set(text_in_words[i: i + SEQUENCE_LEN + 1]).intersection(ignored_words)) == 0:
            sentences.append(text_in_words[i: i + SEQUENCE_LEN])
            next_words.append(text_in_words[i + SEQUENCE_LEN])
        else:
            ignored = ignored + 1
    print('Ignored sequences:', ignored)
    print('Remaining sequences:', len(sentences))

    return sentences, next_words


def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []

    for i in np.random.RandomState(seed=42).permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1. - (percentage_test / 100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))

    print('end: shuffle_and_split_training_set')

    return (x_train, y_train), (x_test, y_test)

In [11]:
corpus_path = 'lyrics.txt'
text_in_words, ignored_words, word_indices, indices_word, words = corpus_to_dictionary(corpus_path)
sequences, next_words = create_and_filter_sequences(text_in_words, ignored_words)
(sentences_train, next_words_train), (sentences_test, next_words_test) = shuffle_and_split_training_set(sequences,
                                                                                                            next_words)

corupus to dict
Corpus length in words: 2835369
Unique words: 78314
Unique words after removing ignored words: 11058
EOF: corpus_to_dictionary()
start: create_and_filter_sequences
Ignored sequences: 1746266
Remaining sequences: 1089073
Shuffling sentences
Size of training set = 1067291
Size of test set = 21782
end: shuffle_and_split_training_set


In [0]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y
        

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
  
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
      seed_index = np.random.randint(len(sequences+sentences_test))
      seed = (sequences+sentences_test)[seed_index]
      sentence=seed
      print('----- diversity:', diversity)

      print('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
      #print(' '.join(sentence))

      for i in range(200):
          x_pred = np.zeros((1, SEQUENCE_LEN))
          for t, word in enumerate(sentence):
              x_pred[0, t] = word_indices[word]

          preds = model.predict(x_pred, verbose=0)[0]
          next_index = sample(preds, diversity)
          next_word = indices_word[next_index]

          sentence = sentence[1:]
          sentence.append(next_word)


          sys.stdout.write(" "+next_word)
          sys.stdout.flush()


#checkpoint = GoogleDriveCheckpoint(file_path, drive, monitor='val_acc', save_best_only=True)
#print_callback = keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)
#early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=50)
#callbacks_list = [checkpoint, print_callback]

In [0]:
file_path = "/content/gdrive/My Drive/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}.h5" % (
    len(words),
    SEQUENCE_LEN,
    MIN_WORD_FREQUENCY
)
checkpoint = GoogleDriveCheckpoint(file_path, drive, monitor='val_acc', save_best_only=True)
print_callback = keras.callbacks.LambdaCallback()
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]


In [0]:
def get_model(words, dropout=0.2):
    model = tf.keras.models.Sequential()
    model.add(Embedding(input_dim=len(words), input_length=SEQUENCE_LEN, output_dim=1024))
    model.add(Bidirectional(LSTM(256, return_sequences=False)))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))

    return model

In [12]:
#tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_address)
#tf.contrib.distribute.initialize_tpu_system(resolver)
#strategy = tf.contrib.distribute.TPUStrategy(resolver)
#with strategy.scope():
model = get_model(words)
# model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
      model,
      strategy=tf.contrib.tpu.TPUDistributionStrategy(
      tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://'    + os.environ['COLAB_TPU_ADDR'])
      )
  )
tpu_model.compile(
        optimizer=tf.train.AdamOptimizer(learning_rate=5e-2),
        loss= tf.keras.losses.sparse_categorical_crossentropy,
        metrics=['accuracy']
    )
tpu_model.summary()
h1=tpu_model.fit_generator(generator(sentences_train, next_words_train, BATCH_SIZE),
                  steps_per_epoch=int(len(sentences_train)/BATCH_SIZE) + 1,
                  epochs=25,
                  callbacks=callbacks_list,
                  validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                  validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Querying Tensorflow master (grpc://10.6.187.130:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 10656900527149095822)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7936460287649914417)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12546962611547536352)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14693880056184474949)
INFO:t

KeyboardInterrupt: ignored

In [0]:
for diversity in [0.2, 0.5, 1.0, 1.2]:
    seed_index = np.random.randint(len(sequences+sentences_test))
    seed = (sequences+sentences_test)[seed_index]
    sentence=seed
    print('----- diversity:', diversity)

    print('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
    #print(' '.join(sentence))

    for i in range(200):
        x_pred = np.zeros((1, SEQUENCE_LEN))
        for t, word in enumerate(sentence):
            x_pred[0, t] = word_indices[word]

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]

        sentence = sentence[1:]
        sentence.append(next_word)


        sys.stdout.write(" "+next_word)
        sys.stdout.flush()
    print()
    print()

----- diversity: 0.2
----- Generating with seed:
"
 we bathe in the blood of the unlucky stiffs 
 keep their eyes tongues and brains in glass cases 
 smear our naked writhing bodies in the grue and"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


----- diversity: 0.5
----- Generating with seed:
"
 
 we make the world go round in tears 
 we are the shadow when faith disappears 
 we are anxiety and fearless scum 
 awaking your sorrow like"

 
 
 
 
 
 
 
 
 
 this 
 
 
 i 
 
 so 
 is 
 
 and 
 
 
 
 
 
 the 
 
 
 the 
 
 the 
 
 
 the this 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 to 
 
 is 
 
 
 
 
 
 
 
 
 
 and 
 
 
 
 
 
 
 the